In [1]:
%cd ~/qsar_flash/

/home/nm_rostislav/qsar_flash


In [2]:
import flash
from torchmetrics import MeanAbsoluteError

from qsar_flash import MolecularGraphRegressor, GraphRegressionData, QM9Property

In [5]:
from torch.nn import functional as F

dataset = QM9Property(property_index=4, root="data/qm9")

datamodule = GraphRegressionData.from_datasets(
    train_dataset=dataset,
    val_split=0.2,
    batch_size=32,
    num_workers=64,
    pin_memory=True,
)

# 2. Build the task
backbone_kwargs = dict(
    hidden_channels=128,
    num_filters=128,
    num_interactions=6,
    num_gaussians=100,
    cutoff=10.,
    max_num_neighbors=32,
)
model = MolecularGraphRegressor(
    backbone="SchNet", metrics=MeanAbsoluteError(), learning_rate=0.0001, pooling_fn="add", optimizer="Adam", loss_fn=F.l1_loss,
    backbone_kwargs=backbone_kwargs
)

In [6]:
# 3. Create the trainer and fit the model
trainer = flash.Trainer(max_epochs=50, gpus=[1])
trainer.fit(model, datamodule=datamodule)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name          | Type                      | Params
------------------------------------------------------------
0 | train_metrics | ModuleDict                | 0     
1 | val_metrics   | ModuleDict                | 0     
2 | test_metrics  | ModuleDict                | 0     
3 | backbone      | SchNetBackbone            | 485 K 
4 | head          | DefaultMolecularGraphHead | 16.6 K
------------------------------------------------------------
502 K     Trainable params
0         Non-trainable params
502 K     Total params
2.010     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]